# Python建模库简介

在本书中，我专注于为使用Python进行数据分析提供编程基础。由于数据分析师和科学家经常报告花费大量时间进行数据整理和准备，因此本书的结构反映了掌握这些技术的重要性。

您用于开发模型的库取决于应用程序。许多统计问题可以通过更简单的技术来解决，例如普通的最小二乘回归，而其他问题则可能需要更高级的机器学习方法。幸运的是，Python已成为实现分析方法的首选语言之一，因此在完成本书后，您可以探索许多工具。

在本章中，我将回顾熊猫的一些功能，这些功能在您与熊猫进行数据争吵以及模型拟合和评分之间来回切换时可能会有所帮助。然后，我将简要介绍两个流行的建模工具包statsmodels和scikit-learn。由于每个项目的规模都足以保证自己拥有一本专门的书，因此，我将不做任何详尽的工作，而是将您带到两个项目的在线文档以及其他一些基于Python的有关数据科学，统计和机器学习的书。

## 13.1 pandas与标准代码之间的接口

模型开发的常见工作流程是使用熊猫进行数据加载和清理，然后再切换到建模库以构建模型本身。模型开发过程的重要部分称为机器学习中的特征工程。这可以描述从原始数据集中提取信息的任何数据转换或分析，这些信息可能在建模环境中有用。我们在本书中探讨的数据聚合和GroupBy工具经常用于功能工程环境中。

尽管“好的”功能工程的详细信息不在本书的讨论范围之内，但我将展示一些方法，以使使用熊猫进行的数据处理和建模之间的切换尽可能轻松。

pandas与其他分析库之间的接触点通常是NumPy数组。要将DataFrame转换为NumPy数组，请使用.values属性：

In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2]})

In [4]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [5]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [6]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

如您可能在前面的章节中所提到的，要转换回DataFrame，可以传递带有可选列名的二维ndarray：

In [7]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])

In [8]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


> .values属性适用于数据同质的情况，例如所有数字类型。如果您具有异构数据，则结果将是Python对象的ndarray：

In [10]:
df3 = data.copy()

In [11]:
df3['string'] = ['a', 'b', 'c', 'd', 'e']

In [12]:
df3

,x0,x1,y,string
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [13]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

对于某些模型，您可能只希望使用列的子集。我建议对值使用loc索引：

In [14]:
model_cols = ['x0', 'x1']

In [15]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

一些库具有对熊猫的本机支持，并且会自动为您完成一些工作：从DataFrame转换为NumPy，并将模型参数名称附加到输出表或Series的列。在其他情况下，您将必须手动执行此“元数据管理”。

在第12章中，我们研究了熊猫的分类类型和pandas.get_dummies函数。假设我们的示例数据集中有一个非数字列：

In [16]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'], categories=['a', 'b'])

In [17]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


如果我们想用虚拟变量替换“类别”列，我们创建虚拟变量，删除“类别”列，然后加入结果：

In [18]:
dummies = pd.get_dummies(data.category, prefix='category')

In [19]:
data_with_dummies = data.drop('category', axis=1).join(dummies)

In [20]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


使用虚拟变量拟合某些统计模型有一些细微差别。当您有不只简单的数字列时，使用Patsy（下一节的主题）可能会更简单且更不易出错。

## 13.2 使用Patsy创建模型描述

Patsy是一个Python库，用于描述统计模型（尤其是线性模型），该模型具有基于字符串的小型“公式语法”，其灵感来自于（但不完全是）与R和S统计编程语言使用的公式语法相同。

Patsy支持在statsmodels中指定线性模型，因此，我将重点介绍一些主要功能来帮助您启动和运行。Patsy的公式是一种特殊的字符串语法，如下所示：

```
    y ~ x0 + x1
```

语法a + b并不意味着将a添加到b，而是这些是为模型创建的设计矩阵中的术语。patsy.dmatrices函数将公式字符串与数据集（可以是DataFrame或数组的dict）一起使用，并为线性模型生成设计矩阵：

In [32]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0, 3.6, 1.3, -2]
})

In [33]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [34]:
import patsy

In [35]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [36]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [37]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

这些Patsy DesignMatrix实例是带有附加元数据的NumPy ndarray：

In [38]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [39]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

您可能想知道“拦截”一词的来源。这是线性模型的惯例，例如普通最小二乘（OLS）回归。您可以通过向模型添加项+ 0来抑制截距：

In [40]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

Patsy对象可以直接传递到numpy.linalg.lstsq之类的算法中，该算法执行普通的最小二乘回归：

In [46]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

/Users/gaussic/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [49]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [50]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [51]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Patsy公式中的数据转换

您可以将Python代码混合到Patsy公式中；在评估公式时，库将尝试查找您在封闭范围内使用的函数：

In [52]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [53]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

一些常用的变量转换包括标准化（均值0和方差1）和居中（减去均值）。 Patsy为此具有内置功能：

In [56]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [57]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

在建模过程中，您可以将模型拟合到一个数据集上，然后根据另一个模型评估该模型。这可能是保留部分，也可能是以后观察到的新数据。在应用中心转换和标准化之类的转换时，使用模型基于新数据形成谓词时应格外小心。这些称为有状态转换，因为在转换新数据集时必须使用统计信息，例如原始数据集的均值或标准差。

patsy.build_design_matrices函数可以使用原始样本数据集中保存的信息将转换应用于新的样本外数据：

In [58]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]
})

In [59]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [60]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

因为在Patsy公式的上下文中加号（+）并不意味着加法，所以当您要按名称从数据集中添加列时，必须将其包装在特殊的I函数中：

In [61]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [62]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

Patsy在patsy.builtins模块中还有其他一些内置的转换。有关更多信息，请参见在线文档。

分类数据具有一类特殊的转换，我将在下面解释。

### 分类数据和 Patsy

可以通过许多不同方式将非数字数据转换为模型设计矩阵。对该主题的完整处理不在本书的范围内，最好与统计学课程一起研究。

当您在Patsy公式中使用非数字术语时，默认情况下会将它们转换为虚拟变量。如果存在截距，将忽略其中一个级别以避免共线性：

In [63]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [65]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [66]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

如果从模型中省略了截距，则每个类别值的列将包括在模型设计矩阵中：

In [67]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [68]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

可以使用C函数将数值列解释为类别：

In [69]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [70]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

当您在模型中使用多个分类项时，事情可能会变得更加复杂，因为您可以包括key1：key2形式的交互项，例如，可用于方差分析（ANOVA）模型中：

In [71]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

In [72]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [73]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [74]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [75]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [76]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

Patsy提供了其他转换分类数据的方法，包括对具有特定顺序的术语进行转换。有关更多信息，请参见在线文档。

## 13.3 statsmodels介绍

statsmodels是一个Python库，用于拟合多种统计模型，执行统计测试以及数据探索和可视化。Statsmodels包含更多的“经典”常客统计方法，而贝叶斯方法和机器学习模型可在其他库中找到。

statsmodels中发现的一些模型包括：

- 线性模型，广义线性模型和鲁棒线性模型
- 线性混合效果模型
- 方差分析（ANOVA）方法
- 时间序列过程和状态空间模型
- 广义矩法

在接下来的几页中，我们将在statsmodels中使用一些基本工具，并探索如何将建模接口与Patsy公式和pandas DataFrame对象一起使用。

### 估计线性模型

stats模型中有几种线性回归模型，从更基本的（例如，普通最小二乘法）到更复杂的（例如，迭代地加权最小二乘）。

statsmodels中的线性模型具有两个不同的主要接口：基于数组和基于公式。这些可以通过以下API模块导入进行访问：

In [77]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

/Users/gaussic/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


为了展示如何使用它们，我们从一些随机数据生成线性模型：

In [78]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

In [79]:
np.random.seed(12345)

In [86]:
N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

在这里，我写下了带有已知参数beta的“真实”模型。在这种情况下，dnorm是用于生成具有特定均值和方差的正态分布数据的辅助函数。 现在我们有了：

In [87]:
X[:5]

array([[ 0.01002535, -0.91965094, -0.1434791 ],
       [ 0.6878291 , -0.31963088,  0.48672384],
       [-0.39923301,  0.91179737, -0.40526637],
       [-0.01434999, -0.2429942 , -0.20757511],
       [ 0.43378812,  1.22311263, -0.22958979]])

In [88]:
y[:5]

array([-0.6576438 ,  0.51064583, -0.22569165, -0.75686704,  0.25164738])

线性模型通常配有一个截距项，就像我们之前在Patsy中看到的那样。sm.add_constant函数可以将拦截列添加到现有矩阵：

In [90]:
X_model = sm.add_constant(X)

In [91]:
X_model[:5]

array([[ 1.        ,  0.01002535, -0.91965094, -0.1434791 ],
       [ 1.        ,  0.6878291 , -0.31963088,  0.48672384],
       [ 1.        , -0.39923301,  0.91179737, -0.40526637],
       [ 1.        , -0.01434999, -0.2429942 , -0.20757511],
       [ 1.        ,  0.43378812,  1.22311263, -0.22958979]])

sm.OLS类可以拟合普通的最小二乘线性回归：

In [92]:
model = sm.OLS(y, X)

模型的拟合方法返回包含估计的模型参数和其他诊断信息的回归结果对象：

In [93]:
results = model.fit()

In [95]:
results.params

array([0.1378007 , 0.25862803, 0.54819111])

结果的汇总方法可以打印详细描述模型诊断输出的模型：

In [96]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.505
Model:                            OLS   Adj. R-squared (uncentered):              0.490
Method:                 Least Squares   F-statistic:                              32.97
Date:                Fri, 17 Jul 2020   Prob (F-statistic):                    8.88e-15
Time:                        22:34:42   Log-Likelihood:                         -23.531
No. Observations:                 100   AIC:                                      53.06
Df Residuals:                      97   BIC:                                      60.88
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

此处为参数名称指定了通用名称x1，x2等。假设所有模型参数都在DataFrame中：

In [97]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

In [99]:
data['y'] = y

In [100]:
data[:5]

,col0,col1,col2,y
0,0.010025,-0.919651,-0.143479,-0.657644
1,0.687829,-0.319631,0.486724,0.510646
2,-0.399233,0.911797,-0.405266,-0.225692
3,-0.014350,-0.242994,-0.207575,-0.756867
4,0.433788,1.223113,-0.229590,0.251647


现在，我们可以使用statsmodels公式API和Patsy公式字符串：

In [102]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

In [103]:
results.params

Intercept   -0.033848
col0         0.142558
col1         0.260027
col2         0.545717
dtype: float64

In [105]:
results.tvalues

Intercept   -1.085211
col0         2.789514
col1         6.170824
col2         7.126278
dtype: float64

观察statsmodels如何以附加了DataFrame列名称的Series返回结果。使用公式和pandas对象时，我们也不需要使用add_constant。

给定新的样本外数据，您可以给定估计的模型参数来计算预测值：

In [109]:
results.predict(data[:5])

0   -0.349852
1    0.246709
2   -0.074831
3   -0.212356
4    0.220744
dtype: float64

您可以探索用于统计模型中线性模型结果的分析，诊断和可视化的许多其他工具。除普通最小二乘法外，还有其他线性模型。

### 估计时间序列过程

statsmodels中的另一类模型用于时间序列分析。其中包括自回归过程，卡尔曼滤波和其他状态空间模型以及多元自回归模型。

让我们用自回归结构和噪声模拟一些时间序列数据：

In [110]:
init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

此数据具有参数为0.8和–0.4的AR（2）结构（两个滞后）。当您拟合一个AR模型时，您可能不知道要包括的滞后项的数量，因此您可以对模型进行一些滞后处理：

In [111]:
MAXLAGS = 5

In [112]:
model = sm.tsa.AR(values)

In [113]:
results = model.fit(MAXLAGS)

结果中的估计参数首先具有截距，然后是前两个滞后的估计：

In [115]:
results.params

array([ 0.00508585,  0.7420653 , -0.36553783, -0.04898946,  0.02224717,
       -0.02366431])

这些模型以及如何解释其结果的更深入的细节超出了本书的范围，但是在statsmodels文档中还有很多发现。